# セグメンテーション

### import

In [11]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

## ファイルを選択

### パラメータ

In [12]:
num = 1
filename = "seg_cow.ply"
name = "seg_cow"

## ファイルを読み込む

In [13]:
def Loading(filename):#PointCloudを読み込んでダウンサンプリング
    print("Loading a point cloud from", filename)
    pcd = o3d.io.read_point_cloud(filename)
    print(pcd)
    pcd = pcd.voxel_down_sample(voxel_size=0.01)
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)
    return points, colors, pcd

In [14]:
load = Loading(filename)

Loading a point cloud from nb368_0028.ply
PointCloud with 368640 points.


## ダウンサンプリング

## DBSCAN

In [15]:
def DBScan(pcd):#DBScanを行う
    labels = np.array(pcd.cluster_dbscan(eps=0.1, min_points=270))
    max_label = labels.max()
    print(len(labels))
    print(f"point cloud has {max_label + 1} clusters")
    seg = plt.get_cmap("tab20")(labels / max(max_label,1))
    seg[labels < 0] = 0
    #セグメンテーションの可視化
    pcd_seg =o3d.geometry.PointCloud()
    pcd_seg.points = o3d.utility.Vector3dVector(pcd.points)
    pcd_seg.colors = o3d.utility.Vector3dVector(seg[:, :3])
    return labels, max_label, pcd_seg

In [16]:
scan = DBScan(load[2])

58109
point cloud has 5 clusters


In [17]:
def Devide(labels, points, colors, d):#セグメンテーションを行う
    p = 0
    count = np.count_nonzero(labels == d)
    print(f"{d+1}番目の点群数:{count}")
    pcd_pt = np.zeros((count, 3), dtype=np.float32)
    pcd_cl = np.zeros((count, 3), dtype=np.float32)
    for k in range(len(labels)):#ここでラベルごとに点群を分けている
        if (labels[k] == d):
            pcd_pt[p] = points[k]
            pcd_cl[p] = colors[k]
            p = p + 1
    #点群を統合している
    pcd_dv =o3d.geometry.PointCloud()
    pcd_dv.points = o3d.utility.Vector3dVector(pcd_pt)
    pcd_dv.colors = o3d.utility.Vector3dVector(pcd_cl)
    return pcd_dv

## 分類した点群を表示

In [18]:
o3d.visualization.draw_geometries([scan[2]])

## 保存するか?

In [19]:
for d in range(0,scan[1]+1):
    devide = Devide(scan[0],load[0],load[1],d)
    o3d.visualization.draw_geometries([devide])
    YorN = input("Y or N?:")
    if YorN == "y":
        s = str(num)
        o3d.io.write_point_cloud(f"{name}_seg{s}.ply", devide)
        print(f"{name}_seg{s}.ply was saved")
        num = num + 1

1番目の点群数:5342
2番目の点群数:25445
nb368_0028_seg1.ply was saved
3番目の点群数:6187
nb368_0028_seg2.ply was saved
4番目の点群数:419
5番目の点群数:299


## PLYファイルの保存

In [20]:
###o3d.io.write_point_cloud(f"{name}_seg{s}.ply", devide)
###このコードで保存している